In [ ]:
from google.colab import drive
drive.mount('/content/drive')#, force_remount=True)

In [ ]:
!pip install sentence_transformers
!pip install evaluate
!pip install bert_score
!pip install gradio==3.48.0
!pip install kaleido

In [ ]:
from sentence_transformers import SentenceTransformer
model_emb = SentenceTransformer('BAAI/bge-large-en-v1.5')

In [ ]:
import pandas as pd
data = pd.read_csv("drive/MyDrive/Сертификат/news-dataset/assets/test.csv",header=None)
data = data.values
for i in range(len(data)):
    data[i][0]=i

In [ ]:
all_embs =  []
from tqdm import tqdm
for el in tqdm(data):
    embs = model_emb.encode([el[2]], normalize_embeddings=True)
    all_embs.append((el[0],embs))

In [ ]:
!pip install chromadb

In [ ]:
!python --version

In [ ]:
!pip install --upgrade --force-reinstall fastapi==0.103.2

In [ ]:
import chromadb

In [ ]:

client = chromadb.Client()

collection = client.create_collection("cos_db", {"hnsw:space": "cosine"})
abc = [el[1][0].tolist() for el in all_embs]
collection.add(
    embeddings=abc,
    metadatas=[{"headline":el} for el in data[:,1].tolist()],
    ids=[str(el[0]) for el in all_embs]
)

In [ ]:
def get_documents(q):
    instruction = "Represent this sentence for searching relevant passages:"
    query = instruction + " " + q
    query = model_emb.encode(query, normalize_embeddings=True)
    results = collection.query(query_embeddings=[query.tolist()],n_results=10)
    return results

In [ ]:
def get_answer(question):
    docs = [data[int(el)][2] for el in get_documents(question)["ids"][0]]
    return docs

In [ ]:
questions = ["Who are the founders of Microsoft?",
             "Who is russia president?",
            "Who is the mobile phone China leader?",
            "Who first developed the theory of evolution",
            'What political party is currently in power in the United States?',
             "Where President Bush meets with Italian Prime Minister Silvio Berlusconi?",
             "What is a record of global sales of mobile telephones?",
             "Who was George Bush?",
             'What is the currency of Japan?',
             'Who wrote the play "Romeo and Juliet"?']

answers = ["Bill Gates and Paul Allen",
           "Vladimir Putin",
          "Samsung",
          "Charles Darwin",
           'The Democratic Party',
           "White House",
           "156 mn.",
           'USA President',
           'yen',
           'William Shakespeare']

In [ ]:
context1 = [get_answer(i) for i in questions]

In [ ]:
get_answer("Who was George Bush?")

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

model_name = "deepset/roberta-base-squad2"

nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)

from evaluate import load
bertscore = load("bertscore")


In [ ]:
bs_all = []
for q, a, index in zip(questions, answers, range(len(answers))):
  QA_input = {'question': q,
             'context': ' '.join(context1[index])}
  res = nlp(QA_input)
  bs = bertscore.compute(predictions=[res['answer']], references=[a], lang="en")
  bs_all.append(bs)

  print(f'Question: {q}\nAnswer: {res["answer"]}\nTrue answer: {a}\nScore: {bs["f1"][0]}\n ')

In [ ]:
f1_scores = [bs['f1'][0] for bs in bs_all]
sum(f1_scores)/len(f1_scores)

In [ ]:
import gradio as gr

def echo(question, history):
    QA_input = {'question': question,
             'context': ' '.join(context1[1])}
    res = nlp(QA_input)
    return res['answer']

demo = gr.ChatInterface(fn=echo, examples=["hello", "hola", "merhaba"], title="Echo Bot")
demo.launch()